# Investments Project (Spring 2024)

**Authors:**
- Marc-Antoine Allard
- Adam Zinebi
- Paul Teiletche
- ...

**DUE Date: June 21 at 23:59**

---
# Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from utils import annualized_metrics

%load_ext autoreload 
%autoreload 2

---
# 6 - Optimal Fund Portfolio Return (STRAT)

*We now assume that you are running a fund that invests its assets under management in 1-month TBills and adds an ‘overlay’ investment in the three strategies (BaB, IV, MoM) targeting an average annual volatility of 10%. Specifically, consider the return to the fund to be $R_{fund}=R_{T−Bill}+c * R_{STRAT}$,where $R_{STRAT}$ is the return to a strategy that combines BaB, IV, and MoM and where $c$ is a constant that you choose so that the average annual volatility $Vol(c * R_{STRAT})=10\%$.*

*Consider three different approaches to combine the three strategies, BaB, IV, MoM to generate $R_{STRAT}$* :
- *Equal weight the strategies.*
- *Risk-Parity based on the rolling window estimate of the strategy returns volatilities.*
- *Mean-variance optimal combination based on the rolling window mean and covariance matrix of the strategy returns.*

*For each of the three approaches to combining the strategies compute the overall mean, standard deviation, and Sharpe ratio of the resulting ‘optimal’ portfolio. Going forward pick the approach with the risk-parity.*

In [ ]:
# Load returns
BaB_returns = ...
IV_returns = ...
MoM_returns = ...
T_bill_returns = ...

data = pd.DataFrame({
    'date': ...,
    'BaB': BaB_returns,
    'IV': IV_returns,
    'MoM': MoM_returns,
    'T_bill': T_bill_returns
})
data.set_index('date', inplace=True)

# Equal weight strategy
def equal_weighted_strategy(data):
    strategy_returns = data[['BaB', 'IV', 'MoM']].mean(axis=1)
    return strategy_returns

# Risk parity strategy
def risk_parity_strategy(data):
    volatilities = data[['BaB', 'IV', 'MoM']].rolling(window=12).std()
    inverse_vols = 1 / volatilities
    weights = inverse_vols.div(inverse_vols.sum(axis=1), axis=0)
    strategy_returns = (weights * data[['BaB', 'IV', 'MoM']]).sum(axis=1)
    return strategy_returns

# Mean-variance optimization strategy
def mean_variance_strategy(data):
    returns = data[['BaB', 'IV', 'MoM']]
    mean_returns = returns.rolling(window=12).mean()
    cov_matrix = returns.rolling(window=12).cov()
    
    def get_optimal_weights(mean_returns, cov_matrix):
        inv_cov = np.linalg.inv(cov_matrix)
        ones = np.ones(len(mean_returns))
        weights = inv_cov.dot(ones) / ones.dot(inv_cov).dot(ones)
        return weights

    optimal_weights = mean_returns.apply(lambda x: get_optimal_weights(x, cov_matrix.loc[x.name]), axis=1)
    strategy_returns = (optimal_weights * returns).sum(axis=1)
    return strategy_returns

# Function to scale returns to achieve target volatility
def scale_to_target_volatility(strategy_returns, target_vol=0.10):
    annual_vol = strategy_returns.std() * np.sqrt(12)
    scale_factor = target_vol / annual_vol
    return strategy_returns * scale_factor

# Calculate strategies
ew_returns = equal_weighted_strategy(data)
rp_returns = risk_parity_strategy(data)
mv_returns = mean_variance_strategy(data)

# Scale strategies to target volatility
scaled_ew_returns = scale_to_target_volatility(ew_returns)
scaled_rp_returns = scale_to_target_volatility(rp_returns)
scaled_mv_returns = scale_to_target_volatility(mv_returns)

# Fund returns
def fund_return(scaled_strategy, T_bill):
    c = 1  # Since we've already scaled the strategy to target volatility
    return T_bill + c * scaled_strategy

fund_equal = fund_return(scaled_ew_returns, data['T_bill'])
fund_risk_parity = fund_return(scaled_rp_returns, data['T_bill'])
fund_mean_variance = fund_return(scaled_mv_returns, data['T_bill'])

# Compute metrics for each strategy
equal_metrics = annualized_metrics(fund_equal)
risk_parity_metrics = annualized_metrics(fund_risk_parity)
mean_variance_metrics = annualized_metrics(fund_mean_variance)

# Display the results
results = pd.DataFrame({
    'Strategy': ['Equal Weight', 'Risk Parity', 'Mean-Variance'],
    'Mean Return': [equal_metrics[0], risk_parity_metrics[0], mean_variance_metrics[0]],
    'Standard Deviation': [equal_metrics[1], risk_parity_metrics[1], mean_variance_metrics[1]],
    'Sharpe Ratio': [equal_metrics[2], risk_parity_metrics[2], mean_variance_metrics[2]]
})

print(results)

# Plot the results
fig, axs = plt.subplots(3, 1, figsize=(10, 15))

metrics_labels = ['Mean Return', 'Standard Deviation', 'Sharpe Ratio']

for i, metric in enumerate(metrics_labels):
    axs[i].bar(results['Strategy'], results[metric])
    axs[i].set_title(metric)
    axs[i].set_ylabel(metric)

plt.tight_layout()
plt.show()
